### To create a df containing all the transaction data

In [11]:
import csv
import os
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np


In [121]:
url = 'https://fantasy.nfl.com/league/6787729/transactions'
page = urlopen(url)
html = page.read()
page.close()
soup1 = bs(html, 'html.parser') 

In [164]:
def soup_cleaner(df):
    trans = []
    row_marker = 0
    for row in df.find_all('tr'):
        #print(row_marker)
        column_marker = 0
        columns = row.find_all('td')
        col_data = []
        for column in columns:
            col_data.append(column.get_text())
            column_marker +=1
        trans.append(col_data)
        row_marker += 1

    return(trans)

In [167]:
url = 'https://fantasy.nfl.com/league/6787729/transactions?offset=41'
page = urlopen(url)
html = page.read()
page.close()
canned = bs(html, 'html.parser') 
soup = canned.find_all('table')[0]
cleansoup = soup_cleaner(soup)
cleansoup;

In [178]:
def CanadasChunkySoup(url, offset,bowlsize):
    counter = 0
    off_num = 0
    while bowlsize >= off_num:
        if counter == 0:
            page = urlopen(url)
        elif counter > 0:
            page = urlopen(url+'?offset='+str(off_num))
        html = page.read()
        page.close()
        canned = bs(html, 'html.parser') 
        soup = canned.find_all('table')[0]
        cleansoup = soup_cleaner(soup)
        if counter == 0: 
            soupbowl = cleansoup
        if counter > 0:
            soupbowl = soupbowl + cleansoup
        counter += 1
        off_num = (offset*counter)+1
    soupbowl = pd.DataFrame(soupbowl)
    return soupbowl

In [179]:
# the last table is 1801 away
tables = CanadasChunkySoup('https://canada.fantasy.nfl.com/league/6787729/transactions', 20,1801)
tables

#### Now that we have the transaction data, lets clean a bit

In [180]:
tables

,0,1,2,3,4,5,6
0,None,None,None,None,None,None,None
1,"Jan 16, 3:49pm",,LM,Jake changed Viewable to Public from 'No' to '...,Jake,None,None
2,"Dec 27, 8:36am",16,Lineup,James Robinson RB - JAX Q,RB,BN,Sam via Mobile Device
3,"Dec 27, 8:36am",16,Lineup,Antonio Gibson RB - WAS,BN,RB,Sam via Mobile Device
4,"Dec 27, 7:09am",16,Lineup,Brandin Cooks WR - HOU,BN,WR,Matt
...,...,...,...,...,...,...,...
2122,"Sep 7, 1:58pm",,LM,Jake changed Waiver Budget to '100',Jake,None,None
2123,"Sep 7, 1:58pm",,LM,Jake changed Waiver Type from 'Resets to Inver...,Jake,None,None
2124,"Sep 7, 1:55pm",,LM,"Jake changed Draft Time to 'Sep 8, 2020 5:30pm...",Jake,None,None
2125,"Sep 7, 1:55pm",,LM,Jake changed Draft Type from 'offline' to 'live',Jake,None,None


In [75]:
trans

,Date,Week,TransType,Player,Status,Team,Owner
0,None,None,None,None,None,None,None
1,"Jan 16, 3:49pm",,LM,Jake changed Viewable to Public from 'No' to '...,Jake,None,None
2,"Dec 27, 8:36am",16,Lineup,James Robinson RB - JAX Q,RB,BN,Sam via Mobile Device
3,"Dec 27, 8:36am",16,Lineup,Antonio Gibson RB - WAS,BN,RB,Sam via Mobile Device
4,"Dec 27, 7:09am",16,Lineup,Brandin Cooks WR - HOU,BN,WR,Matt
5,"Dec 27, 7:09am",16,Lineup,Keenan Allen WR - LAC,WR,BN,Matt
6,"Dec 26, 9:43pm",16,Lineup,Tyler Boyd WR - CIN,WR,BN,f
7,"Dec 26, 9:43pm",16,Lineup,Greg Ward WR - PHI,BN,WR,f
8,"Dec 26, 9:42pm",16,Drop,Jimmy Graham TE - CHI,Tight Ends Spread Sheets,Waivers,f
9,"Dec 26, 9:42pm",16,Add,Greg Ward WR - PHI,Free Agents,Tight Ends Spread Sheets,f


In [ ]:
cleaning = [i.split('<td',1)[1] for i in tags]